In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import zipfile
from zipfile import ZipFile
import os
import json
import pandas as pd
import numpy as np

In [ ]:
# 폴더안에있는 Json파일 전부
def jsonzip(od_path):
    with zipfile.ZipFile(od_path, 'r') as obj:
        obj.namelist()		# zip파일 내의 파일 이름을 가진 list 반환
        obj.extractall("C:/datazip/")

 
def dataframe(od_path) :
    temp = od_path
    path = temp.split(".")[0]
    filename = od_path.split("/")[2].split(".")[0]
    list = os.listdir(path)
    json_file_list = [path+'/'+ file for file in list if file.endswith('.json')]
    
    imagePath = [] #이미지 파일명 저장 리스트
    label=[] #분류 라벨 값 저장 리스트
    points_1_x=[] #boundary box x,y좌표 리스트
    points_1_y=[] #boundary box x,y좌표 리스트
    points_2_x=[] #boundary box x,y좌표 리스트
    points_2_y=[] #boundary box x,y좌표 리스트
    errorFile =[] #에러 파일 검출 리스트
    shapes_list = [] 
    imageHeight = []
    imageWidth = []
    ratio = []

    for i in json_file_list: #json_file_list의 하나의 인덱스 마다 i라는 변수로 사용한다.
        with open(i,'r') as j: #
            try:
                contents=json.loads(j.read())  # open : r - 읽기모드, w-쓰기모드, a-추가모드
                for k in range(len(contents['shapes'])): # contents라는 json객체 안에 여러 객체가 있을 경우를 대비해 반복문안에 shapes의 길이로 반복한다
                                                        # shapes의 내용이 2개면 2번 반복하는 형태 --> 변수는 k사용
                    print(contents['shapes'][k]) # 결과 확인 및 데이터 확인                 
                    shapes_list.append(len(contents['shapes']))
                    imagePath.append((contents['imagePath']).replace('.jpg',"")) # 따로 구별하는 변수가 없어서 이미지 경로에 확장자만 제거하여 사용
                    label.append(contents['shapes'][k]['label']) # label은 분리수거 되는 분류 클래스로 shapes->label 키 값으로 접근
                    points_1_x.append(contents['shapes'][k]['points'][0][0])
                    points_1_y.append(contents['shapes'][k]['points'][0][1])
                    points_2_x.append(contents['shapes'][k]['points'][1][0])
                    points_2_y.append(contents['shapes'][k]['points'][1][1])
                    imageHeight.append(contents['imageHeight'])
                    imageWidth.append(contents['imageWidth'])
            except:
                errorFile.append(i) # 크기가 0이거나, 파일 불러올 때 오류를 유발하는 파일명 errorFile리스트에 담는다.

    # 데이터 프레임 만들기
    # 딕셔너리 형태로 리스트를 묶어서 넣어준다.
    points_1_x=np.round(points_1_x).astype('int')
    points_1_y=np.round(points_1_y).astype('int')
    points_2_x=np.round(points_2_x).astype('int')
    points_2_y=np.round(points_2_y).astype('int')
    for i in range(len(imagePath)) :
        ratio.append((((points_2_x[i] - points_1_x[i])* (points_2_y[i] - points_1_y[i]))/( imageHeight[i] * imageWidth[i]))*100)
        print("Ratio for image", imagePath[i], ":", ratio)

    ratio=np.round(ratio,2)

    data=pd.DataFrame({"imagePath":imagePath,
                        "label":label,
                        "Shape_num":shapes_list,
                        "point1_x":points_1_x,
                        "point1_y":points_1_y,
                        "point2_x":points_2_x,
                        "point2_y":points_2_y,
                        "imageHeight":imageHeight,
                        "imageWidth":imageWidth,
                        "ratio" : ratio
                        })
    data.to_csv(f"{path}/{filename}.csv",index=False)
    return f"{path}/{filename}.csv"

In [ ]:
# 플라스크 실행 코드

app = Flask(__name__)
CORS(app)

# 클릭했을때 반환해주는거잖아
@app.route('/test', methods=['POST'])
def handle_ajax_request():
    od_path = request.form.get('od_path')  # Ajax 요청에서 'od_path' 값을 가져옵니다.
    print(od_path)

    jsonzip(od_path)
    path = dataframe(od_path)
    # 응답을 반환합니다.
    print(path)
    response = {'csv_path': path}
    return jsonify(response)

if __name__ == '__main__':
    app.run()